<a href="https://colab.research.google.com/github/Alias23/IO-PRACTICAS/blob/main/IO_PRACTICA_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from pulp import (
    LpProblem, LpVariable, LpMinimize,
    lpSum, LpStatus, value, GLPK_CMD
)

# Datos
ciudades = ["C1","C2","C3","C4","C5","C6"]
costes = {"C1":30, "C2":45, "C3":40, "C4":50, "C5":80, "C6":90}  # millones €
tiempos = {
    ("C1","C1"):0,  ("C1","C2"):10, ("C1","C3"):13, ("C1","C4"):30, ("C1","C5"):30, ("C1","C6"):20,
    ("C2","C1"):10, ("C2","C2"):0,  ("C2","C3"):25, ("C2","C4"):35, ("C2","C5"):20, ("C2","C6"):10,
    ("C3","C1"):13, ("C3","C2"):25, ("C3","C3"):0,  ("C3","C4"):15, ("C3","C5"):30, ("C3","C6"):20,
    ("C4","C1"):30, ("C4","C2"):35, ("C4","C3"):15, ("C4","C4"):0,  ("C4","C5"):15, ("C4","C6"):25,
    ("C5","C1"):30, ("C5","C2"):20, ("C5","C3"):30, ("C5","C4"):15, ("C5","C5"):0,  ("C5","C6"):14,
    ("C6","C1"):20, ("C6","C2"):10, ("C6","C3"):20, ("C6","C4"):25, ("C6","C5"):14, ("C6","C6"):0
}

BUDGET = 180   # millones €
MAX_TIME = 16  # minutos

# Creamos el modelo
model = LpProblem("Estaciones_Bomberos", LpMinimize)

# Variables binarias
x = {c: LpVariable(f"x_{c}", cat="Binary") for c in ciudades}

# Objetivo: minimizar número de estaciones
model += lpSum(x[c] for c in ciudades), "Min_num_estaciones"

# Restricción de presupuesto
model += lpSum(costes[c] * x[c] for c in ciudades) <= BUDGET, "Presupuesto"

# Cobertura: para cada ciudad j, suma sobre i tales que tiempos[(i,j)] ≤ MAX_TIME
for j in ciudades:
    model += (
        lpSum(
            x[i]
            for i in ciudades
            if tiempos.get((i, j), float('inf')) <= MAX_TIME
        ) >= 1,
        f"Cobertura_{j}"
    )

# Lógica: si C2 y C6, entonces C3
model += x["C2"] + x["C6"] - x["C3"] <= 1, "Cond_C2_C6_a_C3"

# Lógica: si no C2, entonces C5
model += x["C2"] + x["C5"] >= 1, "Cond_noC2_a_C5"

# Resolver con GLPK_CMD (o usa .solve() para CBC)
model.solve(GLPK_CMD(msg=False))
# model.solve()

# Resultados
print("Status:", LpStatus[model.status])
num_est = sum(int(x[c].value()) for c in ciudades)
print("Número de estaciones:", num_est)
print("Ciudades con estación:")
for c in ciudades:
    if x[c].value() > 0.5:
        print(" ", c)
coste_total = value(lpSum(costes[c] * x[c] for c in ciudades))
print("Coste total:", coste_total, "millones €")


Status: Optimal
Número de estaciones: 2
Ciudades con estación:
  C2
  C4
Coste total: 95.0 millones €
